# tsTools Online: MEaSUREs Training Data Collection

In [13]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Label, BoundedIntText,\
                       HTML, Dropdown,Layout, FloatText

import bqplot,datetime, ee, pprint, traitlets, ipyleaflet, geopandas,\
       qgrid, colorsys, sqlite3, os

import dateutil.parser
import IPython.display
import numpy as np
import pandas as pd
import bqplot.pyplot as plt
from dateutil import parser
from interface import interface
from tsmap import Plot_interface
from utils import save_sample


from bqplot.interacts import BrushSelector, BrushIntervalSelector

# Google sheets API
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# PyCCD
import ccd

# Turn off pandas warning
pd.options.mode.chained_assignment = None

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

In [14]:
# Allow Jupyter display to be 100% the width of the browser
display(HTML('<style>.container { width:100% !important; }</style>'))

HTML(value='<style>.container { width:100% !important; }</style>')

In [15]:
%%javascript
// Turn off indefinite output
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [16]:
# Authenticate to the Earth Engine servers, and initialize the ee module.
ee.Initialize()

In [17]:
# Define interface objects
newInterface = interface()
newPlot = Plot_interface(newInterface)

In [18]:
# Non-class buttons and functions. Useful for interacting between objects. 
save_button = widgets.Button(value=False, description='Save', disabled=False,
                button_style='')

def do_save_sample(a):
    # Save sample based on widgets and update the table. 
    # The button cannot pass an argument, so it needs to be done this way.
    save_sample(newInterface, newPlot)
    newPlot.change_table()
    
save_button.on_click(do_save_sample)

In [19]:
# Organize all the pieces

# Right box

# Load data and spreadsheet
load_data = HBox(children=[newInterface.load_button, newInterface.valid_load])
tab0 = VBox(children=[newInterface.spreadsheet, newInterface.spreadName, newInterface.sampleWidget,
                      load_data])

# Land cover information
dropBox1= VBox(children=[newInterface.drop1,
                         newInterface.drop2,
                         newInterface.drop3,
                         newInterface.drop4])

dropBox2= VBox(children=[newInterface.drop5,
                         newInterface.drop6,
                         newInterface.drop7,
                         newInterface.drop8,
                         newInterface.veg_selector])
dropBoxBoth = HBox(children=[dropBox1, dropBox2])
tab1 = VBox(children=[dropBoxBoth])

# Change agents
tab2 = VBox(children=[newInterface.drop9,
                      newInterface.change_selector,
                      newInterface.change_other,
                      newInterface.break_cb,
                      newInterface.break_years])

# Confidence and save
tab3 = VBox(children=[newPlot.error_label,
                      newInterface.confidence,
                      newInterface.years,
                      newInterface.notes,
                      newInterface.valid,
                      save_button])
tab4 = VBox(children=[newPlot.table_widget])

# Tabulate the different boxes
interp_tab = widgets.Tab(children=[tab0, tab1, tab2, tab3, tab4], layout=Layout(width='100%', min_height='270px', max_height='270px'))
interp_tab.set_title(0, 'Load')
interp_tab.set_title(1, 'Land Cover')
interp_tab.set_title(2, 'Change Information')
interp_tab.set_title(3, 'Interpretation')
interp_tab.set_title(4, 'Table')


# Left box
navigate_samples = HBox(children=[newPlot.previous_pt, newPlot.next_pt])
sample_pyccd = HBox([newPlot.pyccd_button, newPlot.pt_message, newPlot.kml_link])
sample_ts_box = VBox([HTML(value="<b>SAMPLE TS</b>"), newPlot.band_selector1, newPlot.ylim, newPlot.xlim, navigate_samples, sample_pyccd])
clicked_ts_box = VBox([HTML(value="<b>CLICKED TS</b>"), newPlot.band_selector2, newPlot.ylim2, newPlot.xlim2, newPlot.pyccd_button2, newPlot.idBox])
ts_tab = HBox([sample_ts_box, clicked_ts_box])
stretch_tab = VBox([Plot_interface.image_band_1, Plot_interface.image_band_2, 
                    Plot_interface.image_band_3, Plot_interface.stretch_min,
                    Plot_interface.stretch_max, Plot_interface.color_check,
                    Plot_interface.clear_layers])
display_tab = widgets.Tab(children=[ts_tab, stretch_tab], layout=Layout(width='50%', min_height='270px', max_height='270px'))

display_tab.set_title(0, 'Controls')
display_tab.set_title(1, 'Display')

# Time series plots
fig = widgets.Tab(children=[newPlot.fig, newPlot.fig3, newPlot.fig2])
fig.set_title(0, 'Sample TS')
fig.set_title(1, 'Sample DOY')
fig.set_title(2, 'Clicked TS')

# Put the boxes together
controls = HBox([display_tab, interp_tab])

# Put the boxes with the time series
tabfig = VBox([controls, fig])

In [20]:
# Display everything
display(newPlot.m)
display(tabfig)


Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{…